In [ ]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import copy
import os
from bisect import bisect_left
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from six.moves import xrange
import enum
import tensorflow as tf

from cleverhans.attacks import FastGradientMethod
from cleverhans.loss import CrossEntropy
from cleverhans.dataset import MNIST, CIFAR10
from cleverhans.model import Model
#from cleverhans.picklable_model import MLP, Conv2D, ReLU, Flatten, Linear, Softmax
from cleverhans.train import train
from cleverhans.utils_tf import batch_eval, model_eval
from cleverhans import serial
from pathlib import Path

from dknn import DkNNModel, get_tensorflow_session, NearestNeighbor
from utils_config import get_config_parameters, dataset_loader, get_model

from utils_kernel import euclidean_kernel, hard_geodesics_euclidean_kernel
from utils_visualization import plot_kernel

In [ ]:
import time

In [ ]:
config = get_config_parameters('../configs/config_mnist.yaml')

In [ ]:
dataset = dataset_loader(config)
x_train, y_train = dataset.get_set('train')
x_test, y_test = dataset.get_set('test')

# Use Image Parameters.
img_rows, img_cols, nchannels = x_train.shape[1:4]
nb_classes = y_train.shape[1]

# Use a holdout of the test set to simulate calibration data for the DkNN.
train_data = x_train
train_labels = np.argmax(y_train, axis=1)
cali_data = x_test[:config['nb_cali']]
y_cali = y_test[:config['nb_cali']]
cali_labels = np.argmax(y_cali, axis=1)
test_data = x_test[config['nb_cali']:]
y_test = y_test[config['nb_cali']:]
x_test = x_test[config['nb_cali']:]

In [ ]:
with get_tensorflow_session(config) as sess:
    with tf.variable_scope('dknn'):
        # Define input TF placeholder.
        x = tf.placeholder(tf.float32, shape=(
          None, img_rows, img_cols, nchannels))
        y = tf.placeholder(tf.float32, shape=(None, nb_classes))

        # Define a model.
        model = get_model(config)
        preds = model.get_logits(x)
        loss = CrossEntropy(model, smoothing=0.)

        # Define the test set accuracy evaluation.
        def evaluate():
            acc = model_eval(sess, x, y, preds, x_test, y_test,
                             args={'batch_size': config['batch_size']})
            print('Test accuracy on test examples: %0.4f' % acc)

        # Train the model
        train_params = {'nb_epochs': config['max_epochs'],
                      'batch_size': config['batch_size'], 'learning_rate': config['learning_rate']}

        data_filepath = "model.joblib"
        path = Path(data_filepath)

        if path.is_file():
            model = serial.load(data_filepath)
        else:
            train(sess, loss, x_train, y_train, evaluate=evaluate,
              args=train_params, var_list=model.get_params())
            serial.save("model.joblib", model)

          # Define callable that returns a dictionary of all activations for a dataset
        def get_activations(data):
            data_activations = {}
            for layer in layers:
                layer_sym = tf.layers.flatten(model.get_layer(x, layer))
                data_activations[layer] = batch_eval(sess, [x], [layer_sym], [data],
                                                   args={'batch_size': config['batch_size']})[0]
            return data_activations

        # Extract representations for the training and calibration data at each layer of interest to the DkNN.
        layers = ['ReLU1', 'ReLU3', 'ReLU5', 'logits']

        #Euclidean DKNN
        dknn = DkNNModel(
        neighbors = config['nb_neighbors'],
        proto_neighbors = config['nb_proto_neighbors'],
        layers=layers,
        get_activations=get_activations,
        train_data=train_data,
        train_labels=train_labels,
        nb_classes=nb_classes,
        backend = config['backend'],
        method='euclidean',
        scope='dknn')
        
        dknn.calibrate(cali_data, cali_labels)
        preds_knn, confs_knn, creds_knn = dknn.predict(x_test)

        start = time.time()
        # Geodesic DKNN
        dknn_geod = DkNNModel(
        neighbors = config['nb_neighbors'],
        proto_neighbors = config['nb_proto_neighbors'],
        layers=layers,
        get_activations=get_activations,
        train_data=train_data,
        train_labels=train_labels,
        nb_classes=nb_classes,
        backend = config['backend'],
        method='geodesic',
        scope='dknn')
        end = time.time()
        print("Time: " + str(end-start))
        dknn_geod.calibrate(cali_data, cali_labels)
        
        preds_geod, confs_geod, creds_geod = dknn_geod.predict(x_test)

In [ ]:
print((preds_knn==np.argmax(y_test,axis=1)).mean())
print((preds_geod==np.argmax(y_test,axis=1)).mean())

In [ ]:
euclidean_matrix = euclidean_kernel(dknn.train_activations['ReLU1'])
max_distance = np.max(euclidean_matrix)+1
euclidean_matrix[euclidean_matrix == 0]=max_distance
plot_kernel(euclidean_matrix)

In [ ]:
geodesic_euclidean_matrix = hard_geodesics_euclidean_kernel(dknn_geod.train_activations['ReLU1'], 5)
max_distance = np.max(geodesic_euclidean_matrix)+1
geodesic_euclidean_matrix[geodesic_euclidean_matrix == 0]=max_distance
plot_kernel(geodesic_euclidean_matrix)

In [ ]:
ks = range(1,70)
same_class_euclidean = np.zeros(len(ks))
same_class_geodesic = np.zeros(len(ks))
for j,k in enumerate(ks):
    acum_euc = 0
    acum_geo = 0
    for i in range(1000):
        euclidean_neighbors_idx = np.argpartition(euclidean_matrix[i,:],k)[:k]
        acum_euc += np.mean(train_labels[i]==train_labels[euclidean_neighbors_idx])

        #geodesic_neighbors_idx = np.argpartition(geodesic_euclidean_matrix[i,:],k)[:k]
        #acum_geo += np.mean(train_labels[i]==train_labels[geodesic_neighbors_idx])
        acum_geo += np.mean(train_labels[i]==train_labels[dknn_geod.query_objects['ReLU1'].train_neighbor_index[i,:k]])
    same_class_euclidean[j] = acum_euc/1000
    same_class_geodesic[j] = acum_geo/1000
    #print(j)

In [ ]:
plt.plot(range(1,70),same_class_euclidean, label='Euclidean', linestyle='--')
plt.plot(range(1,70),same_class_geodesic, label='Geodesic')
plt.legend()
plt.grid()
plt.xlabel('Number of Neighbors')
plt.ylabel('% of NN with same class')
plt.savefig('../results/comformity_comparison_relu1.png')